In [14]:
import pandas as pd
import pandas as pd
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from functions.functions import forward_selection_pvalues, calculate_aic, AIC_forward_selection, remove_high_vif_features
import pandas as pd
import numpy as np

In [15]:
# Load your DataFrame
df_los = pd.read_pickle('./data/los_cat.pkl')
# Drop certain columns as mentioned
df_los = df_los.drop(
    ['altProductLine1', 'altProductLine1SUB', 'payCode', 'LOS', 'AM-LOS', 'GM-LOS', 'patientID'],
    axis=1
)

# Select numerical and categorical columns correctly from df_los
df_numerical = df_los.select_dtypes(include=[np.number])
df_categorical = df_los.select_dtypes(exclude=[np.number])

# Initialize a list to keep the variables with less than 26 unique values
short_cats = [col for col in df_categorical.columns if len(df_los[col].unique()) < 5]
df_categorical = df_categorical[short_cats]
# Concatenate the filtered categorical DataFrame with the numerical DataFrame
# Ensure var_list is used to filter df_categorical correctly
df_los = pd.concat(
    [pd.get_dummies(df_categorical, drop_first=True), df_numerical], axis=1)
df_los = df_los.drop('ethnicity_Not Hispanic or Latino', axis = 1)

In [16]:
import pandas as pd
import numpy as np

# Assuming df_los is your initial DataFrame and is already loaded

# # Select numerical columns
# X_numerical = df_los.select_dtypes(include=[np.number])

# # Convert categorical variables to dummy variables, excluding the first category
# X_categorical = pd.get_dummies(df_los.select_dtypes(exclude=[np.number]), drop_first=True)

# Combine numerical and dummy variables into a single DataFrame
# df = pd.concat([X_numerical, X_categorical], axis=1)

# Define your features (X) by excluding 'LOSDiscrepancyCost' from the combined DataFrame
X = df_los.drop('LOSDiscrepancyCost', axis=1)

# Define the target variable (y) as 'LOSDiscrepancyCost'
y = df_los['LOSDiscrepancyCost']

In [17]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Define a function to calculate VIFs for the given DataFrame
def calculate_vifs(X_df):
    X_df = sm.add_constant(X_df)
    vif_data = pd.DataFrame({
        "Variable": X_df.columns,
        "VIF": [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])]
    })
    return vif_data.drop(index=0)  # Drop the row corresponding to the constant

# VIF threshold
calculate_vifs(X)
# # Iteratively remove variables with VIF above the threshold
# print(f"Processing {X.shape[1]} features...")

# while True:
#     vif_data = calculate_vifs(X)
#     max_vif = vif_data['VIF'].max()
#     if max_vif > vif_threshold:
#         variable_to_drop = vif_data[vif_data['VIF'] == max_vif]['Variable'].values[0]
#         print(f"Dropping '{variable_to_drop}' with VIF={max_vif:.2f}")
#         X = X.drop(columns=[variable_to_drop])
#     else:
#         print("No variables with VIF above the threshold. Stopping iteration.")
#         break

,Variable,VIF
1,ethnicity_Hispanic or Latino,1.012296
2,admitType_cat_Emergency,1.754885
3,admitType_cat_Urgent,1.580933
4,hospital_cat_South Hospital,1.020427
5,sex_cat_Male,1.014600
6,dischargeQTR,1.029282
7,caseMixIndex,2.084988
8,diagPosCnt,1.164317
9,procPosCnt,2.292358
10,totalCharge,2.697747


### Forward Selection P-Values

In [18]:
data = df_los
dependent_var = 'LOSDiscrepancyCost'
predictors = [f'Q("{col}")' if ' ' in col else col for col in X.columns if col != dependent_var]

# dependent_var = 'Y'
# predictors = ['Q("APR DRG")', 'X']  # Note how the variable with a space is quoted
formula = f"{dependent_var} ~ {' + '.join(predictors)}"

forward_p_model = forward_selection_pvalues(data, dependent_var, predictors)
print(forward_p_model.summary())

Adding caseMixIndex with p-value 0.0
Adding admitType_cat_Emergency with p-value 2.0704917408270804e-175
Adding admitType_cat_Urgent with p-value 6.906560474626964e-103
Adding diagPosCnt with p-value 3.560988724504582e-50
Adding procPosCnt with p-value 1.894179712792939e-51
Adding dischargeQTR with p-value 9.489245073222655e-16
Adding totalCharge with p-value 6.438520958001528e-08
Adding Q("hospital_cat_South Hospital") with p-value 6.791067759731851e-06
Adding Q("ethnicity_Hispanic or Latino") with p-value 4.344699574543411e-05
Adding sex_cat_Male with p-value 0.013885250612856075
                            OLS Regression Results                            
Dep. Variable:     LOSDiscrepancyCost   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                  0.747
Method:                 Least Squares   F-statistic:                     2398.
Date:                Thu, 28 Mar 2024   Prob (F-statistic):               0.00
Time:           

### Forward Selection AIC

In [25]:
def calculate_aic(model):
    return model.aic

base_formula = f'{dependent_var} ~ 1'  # Model with only intercept
base_model = ols(base_formula, data=data).fit()
current_aic = calculate_aic(base_model)

predictors = [f'Q("{col}")' if ' ' in col else col for col in X.columns if col != dependent_var]


while True:
    aic_with_candidates = []
    for predictor in predictors:
        # Try adding each remaining predictor to the selected predictors and calculate AIC
        formula = f"{dependent_var} ~ {' + '.join(selected_predictors + [predictor])}"
        model = ols(formula, data=data).fit()
        aic_with_candidates.append((calculate_aic(model), predictor))
    
    # Check if there are no more predictors to evaluate
    if not aic_with_candidates:
        print("No more predictors to evaluate.")
        break

    # Find the predictor that, when added, results in the lowest AIC
    aic_with_candidates.sort()
    best_new_aic, best_new_predictor = aic_with_candidates[0]
    
    # If adding the new variable lowers the AIC, update the model and continue
    if best_new_aic < current_aic:
        predictors.remove(best_new_predictor)
        selected_predictors.append(best_new_predictor)
        current_aic = best_new_aic
        print(f"Added {best_new_predictor}, AIC: {best_new_aic}")
    else:
        # If no improvement, stop the loop
        break

# Final model
final_formula = f"{dependent_var} ~ {' + '.join(selected_predictors)}"
final_model = ols(final_formula, data=data).fit()
print(f"Final model formula: {final_formula}")
print(final_model.summary())


Added caseMixIndex, AIC: 137243.33114628686
Added admitType_cat_Emergency, AIC: 136447.6739802406
Added admitType_cat_Urgent, AIC: 135985.54489221965
Added diagPosCnt, AIC: 135765.53869909473
Added procPosCnt, AIC: 135539.65856684028
Added dischargeQTR, AIC: 135477.0651703751
Added totalCharge, AIC: 135449.80783719243
Added Q("hospital_cat_South Hospital"), AIC: 135431.53292066738
Added Q("ethnicity_Hispanic or Latino"), AIC: 135416.79675854123
Added sex_cat_Male, AIC: 135412.73550308196
No more predictors to evaluate.
Final model formula: LOSDiscrepancyCost ~ caseMixIndex + admitType_cat_Emergency + admitType_cat_Urgent + diagPosCnt + procPosCnt + dischargeQTR + totalCharge + Q("hospital_cat_South Hospital") + Q("ethnicity_Hispanic or Latino") + sex_cat_Male
                            OLS Regression Results                            
Dep. Variable:     LOSDiscrepancyCost   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                 

In [30]:
# Model for prediction
# GET EACH STOCK INDIVIDUALLY
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, mean_squared_error, r2_score

results_dict = {'DT_MSE':[], 'RF_MSE': [], 'LR_MSE':[], 'SVR_MSE': []}

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.3)

# FIT DECISION TREE
# get features needed
max_features = X_train.shape[1]
tree_count = 1000

# set up model
dc_reg = DecisionTreeRegressor(max_features=max_features, random_state=27)
#fit mode
dc_reg.fit(X_train, y_train)
# make predictions
dt_preds = dc_reg.predict(X_test)
# get mse
dt_mse = mean_squared_error(y_test, dt_preds)

# FIT RANDOM FOREST
# set up model
rf_reg = RandomForestRegressor(max_features=max_features, random_state=27, n_estimators=tree_count)
# fit model
rf_reg.fit(X_train, y_train)
# make predictions
rf_preds = rf_reg.predict(X_test)
# get mse
rf_mse = mean_squared_error(y_test, rf_preds)

# FIT LINEAR REGRESSION
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
lr_preds = lr_reg.predict(X_test)
lr_mse = mean_squared_error(y_test, lr_preds)

KeyboardInterrupt: 

In [29]:
# Define a grid of hyperparameters
param_grid = {
    'C': [0.1, 1.0, 10.0],
    'epsilon': [0.01, 0.1, 0.5]
}

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svr_reg = SVR()

# Perform grid search with cross-validation
grid_search = GridSearchCV(svr_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)  

# Retrieve the best parameters
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print("Best parameters:", best_params)
print("Best estimator:", best_estimator)

# FIT SUPPORT VECTOR REGRESSION
# scale for svr
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
svr_reg = SVR(C = best_params['C'], epsilon = best_params['epsilon']) 
svr_reg.fit(X_train_scaled, y_train)
svr_preds = svr_reg.predict(X_test_scaled)
svr_mse = mean_squared_error(y_test, svr_preds)

Best parameters: {'C': 10.0, 'epsilon': 0.1}
Best estimator: SVR(C=10.0)


In [31]:
# APPEND RESULTS
# name
# decesion tree
results_dict['DT_MSE'].append(dt_mse)
# random forest
results_dict['RF_MSE'].append(rf_mse)
# linear regression
results_dict['LR_MSE'].append(lr_mse)
# svr
results_dict['SVR_MSE'].append(svr_mse)

In [36]:
pd.DataFrame(results_dict).T

,0
DT_MSE,4.586321e+05
RF_MSE,2.529429e+05
LR_MSE,9.268402e+05
SVR_MSE,2.839455e+06
